# Homework 4 - Stock Data Analysis

## Problem 1 - IBM Stock Analysis

In [47]:
# Importing data
import pandas as pd

ibm_df = pd.read_csv('ibm-1.csv')
ibm_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,28-Dec-18,109.20,109.75,107.55,108.06,82.36,"5,352,905"
1,27-Dec-18,105.15,108.78,104.66,108.78,82.91,"6,323,698"
2,26-Dec-18,103.25,106.49,101.28,106.49,81.16,"6,951,507"
3,24-Dec-18,105.07,106.12,102.77,102.84,78.38,"3,997,184"
4,21-Dec-18,107.55,110.21,105.58,106.06,80.84,"10,864,279"


In [48]:
# Clean data
ibm_df['Date'] = pd.to_datetime(ibm_df['Date'], format='%d-%b-%y')
ibm_df['Volume'] = ibm_df['Volume'].str.replace(',', '').astype(int)
for col in ['Open', 'High', 'Low', 'Close']:
    ibm_df[col] = ibm_df[col].astype(float)
ibm_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-12-28,109.20,109.75,107.55,108.06,82.36,5352905
1,2018-12-27,105.15,108.78,104.66,108.78,82.91,6323698
2,2018-12-26,103.25,106.49,101.28,106.49,81.16,6951507
3,2018-12-24,105.07,106.12,102.77,102.84,78.38,3997184
4,2018-12-21,107.55,110.21,105.58,106.06,80.84,10864279


In [49]:
# Calculating trade value
ibm_df['Trade Value'] = ibm_df['Close'] * ibm_df['Volume']
ibm_df.head().style.format(
    {col: '${:,.2f}' for col in ['Open', 'High', 'Low', 'Close', 'Trade Value']})

,Date,Open,High,Low,Close,Adj Close,Volume,Trade Value
0,2018-12-28 00:00:00,$109.20,$109.75,$107.55,$108.06,82.360000,5352905,"$578,434,914.30"
1,2018-12-27 00:00:00,$105.15,$108.78,$104.66,$108.78,82.910000,6323698,"$687,891,868.44"
2,2018-12-26 00:00:00,$103.25,$106.49,$101.28,$106.49,81.160000,6951507,"$740,265,980.43"
3,2018-12-24 00:00:00,$105.07,$106.12,$102.77,$102.84,78.380000,3997184,"$411,070,402.56"
4,2018-12-21 00:00:00,$107.55,$110.21,$105.58,$106.06,80.840000,10864279,"$1,152,265,430.74"


In [50]:
# Close price statistics
highest_close = ibm_df['Close'].max()
average_close = ibm_df['Close'].mean()
print('The average closing price was ${0:.2f} with a high of ${1:.2f}'.format(average_close, highest_close))

The average closing price was $137.64 with a high of $161.68


In [51]:
# Days above average close
above_avg = ibm_df[ibm_df['Close'] > average_close]
above_avg.head().style.format(
    {col: '${:,.2f}' for col in ['Open', 'High', 'Low', 'Close', 'Trade Value']})

,Date,Open,High,Low,Close,Adj Close,Volume,Trade Value
50,2018-10-16 00:00:00,$136.64,$139.05,$135.35,$138.74,104.410000,9095493,"$1,261,908,698.82"
55,2018-10-09 00:00:00,$141.57,$141.84,$140.47,$140.76,105.940000,3809950,"$536,288,562.00"
56,2018-10-08 00:00:00,$141.97,$142.51,$140.71,$141.86,106.760000,3388308,"$480,665,372.88"
57,2018-10-05 00:00:00,$144.26,$145.11,$141.37,$142.48,107.230000,4813901,"$685,884,614.48"
58,2018-10-04 00:00:00,$145.02,$146.08,$143.75,$144.66,108.870000,4432843,"$641,255,068.38"


In [52]:
# Convert to Japanese Yen
yen_per_usd = 107
yen_df = ibm_df.copy()

for col in ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Trade Value']:
    yen_df[col] = yen_df[col] * yen_per_usd

# Display in yen format but keep numeric values
yen_df.head().style.format(
    {col: "¥{:,.2f}" for col in ['Open','High','Low','Close','Adj Close','Trade Value']} |
    {'Date': "{:%d-%b-%y}"}
)


,Date,Open,High,Low,Close,Adj Close,Volume,Trade Value
0,28-Dec-18,"¥11,684.40","¥11,743.25","¥11,507.85","¥11,562.42","¥8,812.52",5352905,"¥61,892,535,830.10"
1,27-Dec-18,"¥11,251.05","¥11,639.46","¥11,198.62","¥11,639.46","¥8,871.37",6323698,"¥73,604,429,923.08"
2,26-Dec-18,"¥11,047.75","¥11,394.43","¥10,836.96","¥11,394.43","¥8,684.12",6951507,"¥79,208,459,906.01"
3,24-Dec-18,"¥11,242.49","¥11,354.84","¥10,996.39","¥11,003.88","¥8,386.66",3997184,"¥43,984,533,073.92"
4,21-Dec-18,"¥11,507.85","¥11,792.47","¥11,297.06","¥11,348.42","¥8,649.88",10864279,"¥123,292,401,089.18"


## Problem 2 - IBM Dividend Calculation

In [ ]:
# Organize dataframe
ibm_df = ibm_df.sort_values(['Date'], ascending=[True])
ibm_df = ibm_df.reset_index(drop=True)
ibm_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Trade Value
0,2018-01-02,147.71,148.00,146.79,147.47,107.53,4395815,6.482508e+08
1,2018-01-03,150.42,152.78,149.46,151.52,110.49,9875914,1.496398e+09
2,2018-01-04,152.63,155.18,152.36,154.59,112.72,7903785,1.221846e+09
3,2018-01-05,155.30,155.74,154.02,155.34,113.27,5434807,8.442429e+08
4,2018-01-08,155.51,156.70,154.59,156.28,113.96,5478425,8.561683e+08


In [68]:
# Calculate closing ratios
for i in range(0, len(ibm_df)-1):
    ibm_df.loc[i+1, 'Close Ratio'] = ibm_df.loc[i+1, 'Close'] / ibm_df.loc[i, 'Close']
    ibm_df.loc[i+1, 'Adj Close Ratio'] = ibm_df.loc[i+1, 'Adj Close'] / ibm_df.loc[i, 'Adj Close']
ibm_df.head().style.format(
    {col: '${:,.2f}' for col in ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Trade Value']} |
    {'Close Ratio': '{:.5f}', 'Adj Close Ratio': '{:.5f}'} |
    {'Date': "{:%d-%b-%y}"})

,Date,Open,High,Low,Close,Adj Close,Volume,Trade Value,Close Ratio,Adj Close Ratio
0,02-Jan-18,$147.71,$148.00,$146.79,$147.47,$107.53,4395815,"$648,250,838.05",nan,nan
1,03-Jan-18,$150.42,$152.78,$149.46,$151.52,$110.49,9875914,"$1,496,398,489.28",1.02746,1.02753
2,04-Jan-18,$152.63,$155.18,$152.36,$154.59,$112.72,7903785,"$1,221,846,123.15",1.02026,1.02018
3,05-Jan-18,$155.30,$155.74,$154.02,$155.34,$113.27,5434807,"$844,242,919.38",1.00485,1.00488
4,08-Jan-18,$155.51,$156.70,$154.59,$156.28,$113.96,5478425,"$856,168,259.00",1.00605,1.00609


In [ ]:
# Dividend days (based on arbitrary tolerance)
tol = 1e-4 # comparison tolerance

is_div = ibm_df['Close Ratio'] - ibm_df['Adj Close Ratio'] > tol
dividends = ibm_df[is_div]
dividends.style.format(
    {col: '${:,.2f}' for col in ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Trade Value']} |
    {'Close Ratio': '{:.5f}', 'Adj Close Ratio': '{:.5f}'} |
    {'Date': "{:%d-%b-%y}"})

,Date,Open,High,Low,Close,Adj Close,Volume,Trade Value,Close Ratio,Adj Close Ratio
64,05-Apr-18,$147.65,$148.11,$146.60,$147.26,$108.43,3331928,"$490,659,717.28",0.99946,0.99935
83,02-May-18,$138.11,$138.45,$135.82,$136.19,$100.28,5433865,"$740,038,074.35",0.98247,0.98237
94,17-May-18,$138.15,$138.80,$137.45,$138.15,$102.85,3292494,"$454,858,046.10",0.99913,0.99903
107,06-Jun-18,$137.99,$138.36,$137.23,$138.35,$103.00,3368225,"$465,993,928.75",1.00706,1.00694
130,10-Jul-18,$138.15,$139.19,$137.92,$138.35,$103.00,3950742,"$546,585,155.70",1.00225,1.00214


In [82]:
# Dividend calculation
dividends['Dividend'] = (ibm_df['Close Ratio'] - ibm_df['Adj Close Ratio']) * ibm_df['Close']
# Drop unneeded columns
dividends = dividends.drop(columns = ['Trade Value','High', 'Low'])
dividends.style.format(
    {col: '${:,.2f}' for col in ['Open', 'Close', 'Adj Close', 'Dividend']} |
    {'Close Ratio': '{:.5f}', 'Adj Close Ratio': '{:.5f}'} |
    {'Date': "{:%d-%b-%y}"})

C:\Users\Bryan Keating\AppData\Local\Temp\ipykernel_2524\1450002570.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dividends['Dividend'] = (ibm_df['Close Ratio'] - ibm_df['Adj Close Ratio']) * ibm_df['Close']


,Date,Open,Close,Adj Close,Volume,Close Ratio,Adj Close Ratio,Dividend
64,05-Apr-18,$147.65,$147.26,$108.43,3331928,0.99946,0.99935,$0.02
83,02-May-18,$138.11,$136.19,$100.28,5433865,0.98247,0.98237,$0.01
94,17-May-18,$138.15,$138.15,$102.85,3292494,0.99913,0.99903,$0.01
107,06-Jun-18,$137.99,$138.35,$103.00,3368225,1.00706,1.00694,$0.02
130,10-Jul-18,$138.15,$138.35,$103.00,3950742,1.00225,1.00214,$0.01


In [83]:
import pandas as pd

def calculate_dividends(filename):
    # Load the stock data
    df = pd.read_csv(filename)

    # Clean data
    df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y')
    df['Volume'] = df['Volume'].str.replace(',', '').astype(int)
    for col in ['Open', 'High', 'Low', 'Close', 'Adj Close']:
        df[col] = df[col].astype(float)

    # Compute ratios
    df['Close Ratio'] = df['Close'] / df['Close'].shift(1)
    df['Adj Close Ratio'] = df['Adj Close'] / df['Adj Close'].shift(1)

    # Detect dividend days
    tol = 1e-4
    is_div = df['Close Ratio'] - df['Adj Close Ratio'] > tol
    dividends = df[is_div].copy()

    # Calculate dividends
    dividends['Dividend'] = (df['Close Ratio'] - df['Adj Close Ratio']) * df['Close']

    # Drop unneeded columns
    dividends = dividends.drop(columns=['High', 'Low', 'Volume'])

    return dividends


In [84]:
div_aapl = calculate_dividends("aapl-1.csv")
div_amzn = calculate_dividends("amzn-1.csv")
div_goog = calculate_dividends("goog-1.csv")
div_meta = calculate_dividends("meta-1.csv")
div_msft = calculate_dividends("msft-1.csv")


In [86]:
div_aapl

,Date,Open,Close,Adj Close,Close Ratio,Adj Close Ratio,Dividend
10,2018-12-13,42.62,42.74,40.86,1.033116,1.032861,0.010869
17,2018-12-03,46.12,46.21,44.18,1.046185,1.045928,0.011884
20,2018-11-28,44.18,45.24,43.25,1.007797,1.007689,0.004891
26,2018-11-19,47.50,46.47,44.43,1.050407,1.050106,0.013965
29,2018-11-14,48.47,46.70,44.65,0.975967,0.975743,0.010440
...,...,...,...,...,...,...,...
234,2018-01-24,44.31,43.56,41.03,1.018233,1.018114,0.005169
237,2018-01-19,44.65,44.62,42.03,1.008362,1.008155,0.009198
239,2018-01-17,44.04,44.78,42.18,0.999331,0.999053,0.012445
242,2018-01-11,43.65,43.82,41.28,0.989835,0.989691,0.006327
